## YOLO3

In [8]:
"""
Run a YOLO_v3 style detection model on test images.
"""

import colorsys
import os
import random
from timeit import time
from timeit import default_timer as timer  ### to calculate FPS

import numpy as np
from keras import backend as K
from keras.models import load_model
from PIL import Image, ImageFont, ImageDraw

from yolo3.model import yolo_eval
from yolo3.utils import letterbox_image

class YOLO(object):
    def __init__(self):
        self.model_path = 'model_data/yolo.h5'
        self.anchors_path = 'model_data/yolo_anchors.txt'
        self.classes_path = 'model_data/coco_classes.txt'
        self.score = 0.3
        self.iou = 0.5
        self.class_names = self._get_class()
        self.anchors = self._get_anchors()
        self.sess = K.get_session()
        self.model_image_size = (416, 416) # fixed size or (None, None)
        self.is_fixed_size = self.model_image_size != (None, None)
        self.boxes, self.scores, self.classes = self.generate()

    def _get_class(self):
        classes_path = os.path.expanduser(self.classes_path)
        with open(classes_path) as f:
            class_names = f.readlines()
        class_names = [c.strip() for c in class_names]
        return class_names

    def _get_anchors(self):
        anchors_path = os.path.expanduser(self.anchors_path)
        with open(anchors_path) as f:
            anchors = f.readline()
            anchors = [float(x) for x in anchors.split(',')]
            anchors = np.array(anchors).reshape(-1, 2)
        return anchors

    def generate(self):
        model_path = os.path.expanduser(self.model_path)
        assert model_path.endswith('.h5'), 'Keras model must be a .h5 file.'

        self.yolo_model = load_model(model_path, compile=False)
        print('{} model, anchors, and classes loaded.'.format(model_path))

        # Generate colors for drawing bounding boxes.
        hsv_tuples = [(x / len(self.class_names), 1., 1.)
                      for x in range(len(self.class_names))]
        self.colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
        self.colors = list(
            map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)),
                self.colors))
        #random.seed(10101)  # Fixed seed for consistent colors across runs.
        random.seed(229)  # Fixed seed for consistent colors across runs.
        random.shuffle(self.colors)  # Shuffle colors to decorrelate adjacent classes.
        random.seed(None)  # Reset seed to default.

        # Generate output tensor targets for filtered bounding boxes.
        self.input_image_shape = K.placeholder(shape=(2, ))
        boxes, scores, classes = yolo_eval(self.yolo_model.output, self.anchors,
                len(self.class_names), self.input_image_shape,
                score_threshold=self.score, iou_threshold=self.iou)
        return boxes, scores, classes

    def detect_image(self, image):
        start = time.time()
        count = 0

        if self.is_fixed_size:
            assert self.model_image_size[0]%32 == 0, 'Multiples of 32 required'
            assert self.model_image_size[1]%32 == 0, 'Multiples of 32 required'
            boxed_image = letterbox_image(image, tuple(reversed(self.model_image_size)))
        else:
            new_image_size = (image.width - (image.width % 32),
                              image.height - (image.height % 32))
            boxed_image = letterbox_image(image, new_image_size)
        image_data = np.array(boxed_image, dtype='float32')

        #print(image_data.shape)
        image_data /= 255.
        image_data = np.expand_dims(image_data, 0)  # Add batch dimension.

        out_boxes, out_scores, out_classes = self.sess.run(
            [self.boxes, self.scores, self.classes],
            feed_dict={
                self.yolo_model.input: image_data,
                self.input_image_shape: [image.size[1], image.size[0]],
                K.learning_phase(): 0
            })

        print('Found {} boxes for {}'.format(len(out_boxes), 'img'))

        font = ImageFont.truetype(font='font/FiraMono-Medium.otf',
                    size=np.floor(3e-2 * image.size[1] + 0.5).astype('int32'))
        thickness = (image.size[0] + image.size[1]) // 300

        for i, c in reversed(list(enumerate(out_classes))):
            predicted_class = self.class_names[c]
            box = out_boxes[i]
            score = out_scores[i]

            '''
            if predicted_class != 'car, 3' and predicted_class != 'bus, 6':
                continue            
            '''
                
            if (score <= 0.5) or (predicted_class != 'car, 3' and predicted_class != 'truck, 8'):
                continue

            #label = '{} {:.2f}'.format(predicted_class, score)
            label = '{}, {:.2f}'.format(predicted_class, score)
            draw = ImageDraw.Draw(image)
            label_size = draw.textsize(label, font)

            top, left, bottom, right = box
            top = max(0, np.floor(top + 0.5).astype('int32'))
            left = max(0, np.floor(left + 0.5).astype('int32'))
            bottom = min(image.size[1], np.floor(bottom + 0.5).astype('int32'))
            right = min(image.size[0], np.floor(right + 0.5).astype('int32'))
            
            count += 1
            
            with open('outputEis.txt', 'a') as f:
            #with open('outputEis.csv', 'a') as f:
                #print(label, (left, top), (right, bottom))
                #print("{}, {}, {}, {}, {}".format(label, left, top, right, bottom))
                #print("{}, {}, {}, {}, {}, {}".format(count - 1, label, left, top, right, bottom), file=f)
                print("{}, {}, {}, {}, {}, {}".format(len(out_boxes), label, left, top, right, bottom), file=f)                
            #f.close()

            if top - label_size[1] >= 0:
                text_origin = np.array([left, top - label_size[1]])
            else:
                text_origin = np.array([left, top + 1])

            # My kingdom for a good redistributable image drawing library.
            for i in range(thickness):
                draw.rectangle(
                    [left + i, top + i, right - i, bottom - i],
                    outline=self.colors[c])
            draw.rectangle(
                [tuple(text_origin), tuple(text_origin + label_size)],
                fill=self.colors[c])
            draw.text(text_origin, label, fill=(0, 0, 0), font=font)
            del draw

        end = time.time()
        #print(end - start)
        return image

    def close_session(self):
        self.sess.close()

## Detect Video

In [9]:
def detect_video(yolo, video_path):
    import cv2
    vid = cv2.VideoCapture(video_path)
    
    print("Video Dim: {}  {}  {}".format(vid.get(3), 'X', vid.get(4)))

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    #out = cv2.VideoWriter('test_python_yolo3.avi',fourcc, 30.0, (1920,1080))
    out = cv2.VideoWriter('test_python_yolo3.avi',fourcc, 30.0, (960,540))      # resize image half
    
    if not vid.isOpened():
        raise IOError("Couldn't open webcam or video")
    accum_time = 0
    curr_fps = 0
    fps = "FPS: ??"
    prev_time = timer()
    start4Total = timer()
    cnt = 0
    while True:
        return_value, frame = vid.read()
        
        if return_value == True:
        
            cnt += 1
            # Re-size
            frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)  # resize image half
            # Re-size
            image = Image.fromarray(frame)
            image = yolo.detect_image(image)
            result = np.asarray(image)
            curr_time = timer()
            exec_time = curr_time - prev_time
            prev_time = curr_time
            accum_time = accum_time + exec_time
            curr_fps = curr_fps + 1
            if accum_time > 1:
                accum_time = accum_time - 1
                fps = "FPS: " + str(curr_fps)
                curr_fps = 0
            
            print('Frame Number {}:'.format(cnt - 1))
            
            cv2.putText(result, text=fps, org=(3, 15), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=0.50, color=(255, 0, 0), thickness=2)
            cv2.namedWindow("result", cv2.WINDOW_NORMAL)
            out.write(result)
            cv2.imshow("result", result)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
            
    print('++++++++ Video End ++++++++')
    
    vid.release()
    out.release()
    cv2.destroyAllWindows()
    
    print('\nTotal elapsed time = ' + str(timer() - start4Total) + ' s\n')

    #yolo.close_session()

## Downsample

In [20]:
def detect_video(yolo, video_path):
    import cv2
    vid = cv2.VideoCapture(video_path)
    print("Video Dim: {}  {}  {}".format(vid.get(3), 'X', vid.get(4)))

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('test_python_yolo3.avi',fourcc, 30.0, (1920,1080))
    #out = cv2.VideoWriter('test_python_yolo3.avi',fourcc, 30.0, (960,540))      # resize image half
    i = 0
    frame_rate_divider = 3
    
    if not vid.isOpened():
        raise IOError("Couldn't open webcam or video")
    accum_time = 0
    curr_fps = 0
    fps = "FPS: ??"
    prev_time = timer()
    start4Total = timer()
    cnt = 0
    while True:
        return_value, frame = vid.read()
        
        if return_value == True:
            if i % frame_rate_divider == 0:
                
                cnt += 1
                # Re-size
                #frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)  # resize image half
                # Re-size
                image = Image.fromarray(frame)
                image = yolo.detect_image(image)
                result = np.asarray(image)
                curr_time = timer()
                exec_time = curr_time - prev_time
                prev_time = curr_time
                accum_time = accum_time + exec_time
                curr_fps = curr_fps + 1
                if accum_time > 1:
                    accum_time = accum_time - 1
                    fps = "FPS: " + str(curr_fps)
                    curr_fps = 0
            
                print(cnt-1)
            
                cv2.putText(result, text=fps, org=(3, 15), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=0.50, color=(255, 0, 0), thickness=2)
                cv2.namedWindow("result", cv2.WINDOW_NORMAL)
                out.write(result)
                cv2.imshow("result", result)
                i += 1
            else:
                i += 1
                
                
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
            
    print('++++++++ Video End ++++++++')
    
    vid.release()
    out.release()
    cv2.destroyAllWindows()
    
    print('\nTotal elapsed time = ' + str(timer() - start4Total) + ' s\n')

    #yolo.close_session()

## Detect Video

In [10]:
#from yolo import YOLO
#from yolo import detect_video

if __name__ == '__main__':
    video_path='/media/cfchen/956df7bc-562e-4f24-8339-fd0b67f98888/Downloaded/VideosHPB/IMAG0011.mp4'
    #video_path='/media/cfchen/956df7bc-562e-4f24-8339-fd0b67f98888/Downloaded/VideosHPB/peoplecars.mp4'
    detect_video(YOLO(), video_path)


model_data/yolo.h5 model, anchors, and classes loaded.
Video Dim: 1920.0  X  1080.0
Found 9 boxes for img
Frame Number 0:
Found 9 boxes for img
Frame Number 1:
Found 11 boxes for img
Frame Number 2:
Found 12 boxes for img
Frame Number 3:
Found 9 boxes for img
Frame Number 4:
Found 8 boxes for img
Frame Number 5:
Found 8 boxes for img
Frame Number 6:
Found 9 boxes for img
Frame Number 7:
Found 8 boxes for img
Frame Number 8:
Found 8 boxes for img
Frame Number 9:
Found 8 boxes for img
Frame Number 10:
Found 9 boxes for img
Frame Number 11:
Found 8 boxes for img
Frame Number 12:
Found 7 boxes for img
Frame Number 13:
Found 7 boxes for img
Frame Number 14:
Found 8 boxes for img
Frame Number 15:
Found 8 boxes for img
Frame Number 16:
Found 8 boxes for img
Frame Number 17:
Found 7 boxes for img
Frame Number 18:
Found 8 boxes for img
Frame Number 19:
Found 8 boxes for img
Frame Number 20:
Found 7 boxes for img
Frame Number 21:
Found 7 boxes for img
Frame Number 22:
Found 7 boxes for img
Frame

Found 5 boxes for img
Frame Number 208:
Found 5 boxes for img
Frame Number 209:
Found 4 boxes for img
Frame Number 210:
Found 5 boxes for img
Frame Number 211:
Found 5 boxes for img
Frame Number 212:
Found 3 boxes for img
Frame Number 213:
Found 3 boxes for img
Frame Number 214:
Found 4 boxes for img
Frame Number 215:
Found 4 boxes for img
Frame Number 216:
Found 4 boxes for img
Frame Number 217:
Found 4 boxes for img
Frame Number 218:
Found 4 boxes for img
Frame Number 219:
Found 4 boxes for img
Frame Number 220:
Found 3 boxes for img
Frame Number 221:
Found 3 boxes for img
Frame Number 222:
Found 4 boxes for img
Frame Number 223:
Found 3 boxes for img
Frame Number 224:
Found 4 boxes for img
Frame Number 225:
Found 4 boxes for img
Frame Number 226:
Found 4 boxes for img
Frame Number 227:
Found 4 boxes for img
Frame Number 228:
Found 4 boxes for img
Frame Number 229:
Found 4 boxes for img
Frame Number 230:
Found 3 boxes for img
Frame Number 231:
Found 5 boxes for img
Frame Number 232:
